In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [4]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup, AutoConfig
import torch
import torch.nn as nn
from torch.utils.data import Sampler, Dataset, DataLoader
from IPython.display import display
from accelerate import Accelerator
from tqdm import tqdm
import random
import copy
import os
import multiprocessing
from sklearn.model_selection import StratifiedKFold
# import more_itertools
from sklearn.metrics import f1_score

import string
from bs4 import BeautifulSoup
import re

# Data preprocess

In [5]:
train_raw = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [9]:
train_no_duplicates = train_raw.drop_duplicates('text')

In [10]:
# remove contradictory
duplicates_df = train_raw[train_raw.text.duplicated(keep=False)].sort_values('text')
contradictory_tweets = set()
for tweet in list(duplicates_df.text):
    if len(set(duplicates_df[duplicates_df['text'] == tweet].target)) > 1:
        contradictory_tweets.add(tweet)

contradictory_tweets = list(contradictory_tweets)
print(len(contradictory_tweets))

18


In [11]:
filtered_df = train_no_duplicates[~train_no_duplicates['text'].isin(contradictory_tweets)]
# filtered_df = filtered_df.drop(columns=['id'])

In [12]:
filtered_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
def combine_columns(row):
    values = [f"{col}: {str(row[col])}" for col in row.index[1:-2] if pd.notnull(row[col])]
    return ' '.join(values)

In [14]:
# Combine the three columns into a single column
filtered_df['combined'] = filtered_df.apply(combine_columns, axis=1)
filtered_df['final_text'] =  filtered_df['text']+' '+ filtered_df['combined']
filtered_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,keyword,location,text,target,combined,final_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,,Our Deeds are the Reason of this #earthquake M...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,,Forest fire near La Ronge Sask. Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,,All residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,,Just got sent this photo from Ruby #Alaska as ...
...,...,...,...,...,...,...,...
7604,10863,NaN,NaN,#WorldNews Fallen powerlines on G:link tram: U...,1,,#WorldNews Fallen powerlines on G:link tram: U...
7605,10864,NaN,NaN,on the flip side I'm at Walmart and there is a...,1,,on the flip side I'm at Walmart and there is a...
7606,10866,NaN,NaN,Suicide bomber kills 15 in Saudi security site...,1,,Suicide bomber kills 15 in Saudi security site...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,,Two giant cranes holding a bridge collapse int...


In [15]:
#test set
filtered_test = test.copy()
filtered_test['combined'] = filtered_test.apply(combine_columns, axis=1)
filtered_test['final_text'] =  filtered_test['text']+' '+ filtered_test['combined']
filtered_test

,id,keyword,location,text,combined,final_text
0,0,NaN,NaN,Just happened a terrible car crash,,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,,MEG issues Hazardous Weather Outlook (HWO) htt...


# DataLoader

In [16]:
device = "cuda:0"
model_checkpoint = "bert-large-uncased"

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [19]:
class MyDataset(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, target='target', max_length = 256):
#         text = dataframe[text_column].values.tolist()
#         tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=max_length, return_tensors='pt')
        self.data = []
        
        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
#             self.data.append((torch.tensor(row['id']).to(device) ,(tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device))))
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append(((tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device)), torch.tensor(row[target]).to(device)))
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    
    def train_valid_split(self, train_fraction=.8, shuffle=True):
        num_train_examples = int(len(self) * train_fraction)
        train_dataset = copy.deepcopy(self)
        
        if shuffle:
            random.shuffle(train_dataset.data)
        
        valid_dataset = copy.deepcopy(train_dataset)
        train_dataset.data = train_dataset.data[:num_train_examples]
        valid_dataset.data = valid_dataset.data[num_train_examples:]
        
        return train_dataset, valid_dataset

In [20]:
# Create an instance of your custom Dataset class
dataset = MyDataset(filtered_df, 'final_text', tokenizer)
train_dataset, valid_dataset = dataset.train_valid_split()

100%|████████████████████████████| 7485/7485 [00:09<00:00, 821.18it/s]


In [23]:
class MyDatasetTest(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, max_length = 256):
#         text = dataframe[text_column].values.tolist()
#         tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=max_length, return_tensors='pt')
        self.data = []
        
        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
#             self.data.append((torch.tensor(row['id']).to(device) ,(tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device))))
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
#             self.data.append(((tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device)), torch.tensor(row[target]).to(device)))
            self.data.append((torch.tensor(row['id']).to(device) ,(tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device))))
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [24]:
dataset_test = MyDatasetTest(filtered_test, 'final_text', tokenizer)

100%|███████████████████████████| 3263/3263 [00:01<00:00, 1701.45it/s]


In [15]:
# tokenizer.decode(train_dataset[100][1][0])

In [25]:
# Create a DataLoader
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Set up BERT

In [26]:
class FinetuneClassifier(nn.Module):
    def __init__(self, model=model_checkpoint, classes=2, head_dropout=0.2):
        super().__init__()
        
        self.model = AutoModel.from_pretrained(model)
        
        self.project = torch.nn.Sequential(
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(1024, 1024),
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(1024,classes) # projection
        )

    def forward(self, input_ids, attention_mask=None):
        
        res = self.model.forward(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        res = res[0]
        res = res[:,0,:] # encoding for <s> token
        res = self.project(res)
        return res
    
    def parameters_num(self):
        return sum(p.numel() for p in self.parameters())

In [27]:
model = FinetuneClassifier(head_dropout=.2)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
model.forward(torch.tensor([1,2,33,2])[None])

tensor([[-0.3648, -0.0454]], grad_fn=<AddmmBackward0>)

In [28]:
model.to(device)

FinetuneClassifier(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elem

# Finetuning

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [28]:
a = next(iter(train_dataloader))
a

[[tensor([[  101, 13215,  1999,  ...,     0,     0,     0],
          [  101,  1030,  2522,  ...,     0,     0,     0],
          [  101,  1030,  1051,  ...,     0,     0,     0],
          ...,
          [  101,  1030,  3915,  ...,     0,     0,     0],
          [  101,  1030, 26146,  ...,     0,     0,     0],
          [  101,  2256, 13044,  ...,     0,     0,     0]], device='cuda:0'),
  tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')],
 tensor([0, 0, 1, 0, 0, 1, 0, 0], device='cuda:0')]

In [30]:
def train(model, 
          train_dataloader, 
          valid_dataloader, 
          steps, 
          optimizer,
          blind_steps=None,
          loss_fn=torch.nn.BCELoss(),
          main_metric=('f1', f1_score), 
          additional_metrics=[],
          filepath='model_sd.pt',
          load_best=True,
          scheduler=None,
          losses_dict=None):
    
    if blind_steps == None:
        blind_steps = len(train_dataloader) // 4
    
    def evaluate():  # the first score returned is the main
        model.eval()
        
        y_trues = []
        y_hats = []
        
        loss = 0
        k = 0
        
        with torch.no_grad():
            for batch in valid_dataloader:
                
                (ids, mask), y_true = batch
                ids, mask = ids.to(device), mask.to(device)
                y_true = y_true.to(device)
                hots = torch.nn.functional.one_hot(y_true, 2).to(device, torch.float)
                y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

                loss += float(loss_fn(y_hat, hots))
                k += 1
                
                for i in range(y_true.shape[0]):
                    y_trues.append(int(y_true[i]))
                    y_hats.append(1 if y_hat[i][0] < y_hat[i][1] else 0)
        
        scores = [(main_metric[0], main_metric[1](y_trues, y_hats))]
        
        for metric in additional_metrics:
            scores.append((metric[0], metric[1](y_trues, y_hats)))        
        
        model.train()
        return scores + [('valid_loss', loss/k)]
    
    
    def render_scores(scores, step, best=None):
        print('{:05d} steps'.format(step), end=' ')
        
        for score in scores:
            print("| {}: {:.3f}".format(*score), end=' ')
            
        if best != None:
            print('| best_score: {:.3f}'.format(best))
            
    
    # initial scores
    scores = evaluate()
    render_scores(scores, 0)
    best_score = scores[0][1]
    torch.save(model.state_dict(), filepath)
    
    # logs
    if losses_dict != None:
        losses_dict['train_loss'] = []
        losses_dict['valid_loss'] = []
        losses_dict[main_metric[0]] = []
    
    epoch_loss = 0
    k = 0
    
    train_iter = iter(train_dataloader)
    model.train()
    
    for step in tqdm(range(steps)):
        
        # retrieving a batch
        try:
            batch = next(train_iter)
        except:
            train_iter = iter(train_dataloader)
            batch = next(train_iter)

        (ids, mask), y_true = batch
        ids, mask = ids.to(device), mask.to(device)
        y_true = y_true.to(device)

        # prediction
        y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)
        hots = torch.nn.functional.one_hot(y_true, 2).to(device, torch.float)
        loss = loss_fn(y_hat, hots).to(device)
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if scheduler != None:
            scheduler.step()
            
        epoch_loss += float(loss)
        k+=1
        
        # evaluation
        if (step + 1) % blind_steps == 0:
            scores = evaluate() + [('train_loss', epoch_loss/k)]
            
            if losses_dict != None:
                losses_dict['valid_loss'].append(float(scores[-2][1]))
                losses_dict['train_loss'].append(float(scores[-1][1]))
                losses_dict[main_metric[0]].append(float(scores[0][1]))
            
            if scores[0][1] > best_score:
                best_score = scores[0][1]
                torch.save(model.state_dict(), filepath)
                
            render_scores(scores, step + 1, best=best_score)
            epoch_loss = 0
            k = 0
                
    if load_best:
        model.load_state_dict(torch.load(filepath))

In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-6, weight_decay=2e-3)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=500) 
logs_dict = {}

In [32]:
train(
      model, 
      train_dataloader, 
      valid_dataloader, 
      2000, 
      optimizer, 
      blind_steps=100, 
      additional_metrics=[('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)],
      losses_dict=logs_dict,
      scheduler=scheduler
     )

00000 steps | f1: 0.590 | precision: 0.420 | recall: 0.989 | accuracy: 0.426 | valid_loss: 0.711 

  5%|▌         | 100/2000 [01:59<7:39:25, 14.51s/it]

00100 steps | f1: 0.646 | precision: 0.788 | recall: 0.547 | accuracy: 0.749 | valid_loss: 0.573 | train_loss: 0.641 | best_score: 0.646


 10%|█         | 200/2000 [03:58<7:14:18, 14.48s/it]

00200 steps | f1: 0.679 | precision: 0.811 | recall: 0.584 | accuracy: 0.770 | valid_loss: 0.517 | train_loss: 0.542 | best_score: 0.679


...

01800 steps | f1: 0.796 | precision: 0.819 | recall: 0.774 | accuracy: 0.834 | valid_loss: 0.408 | train_loss: 0.325 | best_score: 0.806


 95%|█████████▌| 1900/2000 [37:15<23:57, 14.38s/it]

01900 steps | f1: 0.806 | precision: 0.790 | recall: 0.822 | accuracy: 0.834 | valid_loss: 0.436 | train_loss: 0.349 | best_score: 0.806


100%|██████████| 2000/2000 [39:11<00:00,  1.18s/it]

02000 steps | f1: 0.799 | precision: 0.828 | recall: 0.771 | accuracy: 0.838 | valid_loss: 0.398 | train_loss: 0.347 | best_score: 0.806


# Evaluate

In [31]:
model.load_state_dict(torch.load('/kaggle/input/model/model_sd.pt'))

<All keys matched successfully>

In [32]:
def evaluate(model, valid_dataloader, metrics=[('f1', f1_score),('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)]):
    model.eval()

    y_trues = []
    y_hats = []

    with torch.no_grad():
        for batch in valid_dataloader:

            (ids, mask), y_true = batch
            y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

            for i in range(y_true.shape[0]):
                y_trues.append(int(y_true[i]))
                y_hats.append(1 if y_hat[i][0] < y_hat[i][1] else 0)

    scores = []

    for metric in metrics:
        scores.append((metric[0], metric[1](y_trues, y_hats)))        

    return scores

In [33]:
scores = evaluate(model, valid_dataloader)
print(scores)

[('f1', 0.8713178294573644), ('precision', 0.8740279937791602), ('recall', 0.8686244204018547), ('accuracy', 0.8891115564462257)]


In [34]:
predictions_df = pd.DataFrame()
for i, (ids, mask) in tqdm(dataset_test):
    pred = model(input_ids=ids[None], attention_mask=mask[None])[0]
    y_hat = 1 if pred[0] < pred[1] else 0
    r = [int(i), y_hat]
    predictions_df = pd.concat([predictions_df, pd.DataFrame(np.array(r)[None,:], columns=['id', 'target'])])

100%|██████████| 3263/3263 [01:53<00:00, 28.76it/s]


In [35]:
predictions_df

,id,target
0,0,1
0,2,1
0,3,1
0,9,1
0,11,1
...,...,...
0,10861,0
0,10865,1
0,10868,1
0,10874,1


In [39]:
predictions_df.to_csv('submission_final.csv', index=False)

In [ ]:
# def finetune_epoch(clf: FinetuneClassifier,
#                    train_dataset,
#                    batch_size=16,
#                    lr=1e-4,
#                    loss_lambda = 0.25,
#                    device="cuda:0"):
#     clf.train()
#     loader = DataLoader(train_dataset, batch_size, drop_last=True)
#       # train_features, train_labels = next(iter(loader))
#       # print(f"Feature batch shape: {train_features.size()}")
#       # print(f"Labels batch shape: {train_labels.size()}")

#     params_to_update = []
#     for name,param in clf.named_parameters():
#         if param.requires_grad == True:
#             params_to_update.append(param)

#     optimizer = torch.optim.Adam(params_to_update, lr=lr, eps=1e-08)
#     criterion = nn.CrossEntropyLoss(ignore_index=-1000).to(device)

#     idx = 0

#     # Train loop.
#     optimizer.zero_grad()
#     logits_dep = clf(tokens)
    
#     # print(logits_dep.size())

#     total_loss_dep = 0

#     for i in range(batch_size):
#         id = batch['id'][i].to(device)

#         dep_label = train_data_dict['dep_label'][id]
#         rel_pos = train_data_dict['rel_pos'][id]
#         dep_label_idx = dep_to_index(dep_label)
#         rel_pos_idx = rel_pos_to_index(rel_pos)

#         UD_tokenized = convert_token_to_UD(batch['text'][i], len(dep_label_idx))

#         target_len = len(logits_dep[i])
#         dep_idx = [-1000]*target_len
#         rel_idx = [-1000]*target_len

#         k=0
#         for j in UD_tokenized:
#         # if j in UD_tokenized:
#         # print(j)
#         # print(len(dep_label_idx))
#         # print(len(dep_idx))
#         dep_idx[j] = dep_label_idx[k]
#         rel_idx[j] = rel_pos_idx[k]
#         k+=1

#         # print(rel_pos)
#         # print(rel_pos_idx)
#         # print(rel_idx)

#         torch_dep_idx = torch.tensor(dep_idx).to(device)
#         torch_rel_idx = torch.tensor(rel_idx).to(device)


#         # print(torch_dep_idx.size())
#         # print(torch_rel_idx.size())
#         # print(logits_dep[i].size())
#         # print(logits_rel[i])
#         # # print(logits_dep[i][focus, :])
#         # # print(dep_label_idx.size())

#         loss_dep = criterion(logits_dep[i], torch_dep_idx)
#         total_loss_dep += loss_dep
#         loss_rel = criterion(logits_rel[i], torch_rel_idx)
#         total_loss_rel += loss_rel

#     loss = (loss_lambda*total_loss_rel+(1-loss_lambda) * total_loss_dep)
#     loss.backward()
#     # Have gradients at this point.
#     nn.utils.clip_grad_norm_(clf.parameters(), max_norm=1.0, norm_type=2)
#     optimizer.step()

#     if idx % 50 == 0:
#       print(loss.item())
#     idx += 1